In [ ]:
import operator
import datetime as dt
from a1chemy.util.option_greeks_calculator import merton, euro_implied_vol
import a1chemy.util.option_greeks_calculator as gbs
import numpy as np

class Instrument(object):
    def __init__(self, price) -> None:
        self.price = price


class Option(object):
    def __init__(self, **kargs):
        self.date = kargs['date']
        self.strike = kargs['strike']
        self.price = kargs['price']
        self.option_type = kargs['option_type']
        self.instrument = kargs['instrument']
        self.iv = kargs.get('iv')
        self.delta = kargs.get('delta')
        self.gamma = kargs.get('gamma')
        self.theta = kargs.get('theta')
        self.vega = kargs.get('vega')
        self.rho = kargs.get('rho')

    def recalculate_greeks(self, r, q):
        current_price = self.instrument.price
#         t = float(np.busday_count( datetime.date.today(), self.date))/256
        t = 0.030136986301369864
        self.iv = euro_implied_vol(self.option_type, fs=current_price, x=self.strike, t=t, r=r, q=q, cp=self.price)
        option_price, self.delta, self.gamma, self.theta, self.vega, self.rho = merton(self.option_type, fs=current_price, x=self.strike, t=t, r=r, q=q, v=self.iv)
        
gbs._DEBUG=True
i = Instrument(price=3.4340)
o = Option(option_type='c', date = dt.date.today(), instrument=i, strike=3.5, price=0.0204)

o.recalculate_greeks(r=0.02, q=0.01485148514851485)
print(o.iv)
print(o.delta)
print(o.gamma)
print(o.theta)
print(o.vega)
print(o.rho)

In [ ]:
from a1chemy.common import *


oo = option_map.get_option(expiry_date_list[1], 5.5, 'c')
print('origin option Data, price:{}, {}'.format(oo.price, oo.toString()))
print('------------------')

new_underlying = Underlying(price=5.3030)
bs_config = OptionConfig(interest_rate=0.029120, pricing_model=black_scholes)

def to_cash_greeks(g):
    return g*10000/new_underlying.price
new_option = Option(underlying=new_underlying, 
                    config=bs_config, 
                    price=oo.price, 
                    option_type=oo.option_type, 
                    time_to_expiry=oo.time_to_expiry, 
                    strike=oo.strike, 
                    redefine_greeks=True)
print(new_option.toString())
print('delta:{} gamma:{} theta:{} vega:{}'.format(to_cash_greeks(new_option.delta), to_cash_greeks(new_option.gamma), to_cash_greeks(new_option.theta), to_cash_greeks(new_option.vega)))
